# Purpose
This is a version of the Claude_Researcher_Google_API.ipynb notebook that prioritizes cheaper API usage.

Prompts have been minimized and made more efficient for use with Haiku to exclusively perform both research and reporting tasks.

Costs average $0.02-0.03 cents per research assignment, versus the $0.10-0.20 of some common model combinations in the other book.

In [ ]:
!pip install google-api-python-client
!pip install anthropic
from googleapiclient.discovery import build
import concurrent.futures
import anthropic
import requests
import json
import ast
import re
import os

# SET ENVIRONMENT VARIABLES ON YOUR PC OR REPLACE THE OS.GETEMV() WITH YOUR API KEYS AS STRINGS
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key
GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

client = anthropic.Anthropic()

In [ ]:
def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string
def generate_text(prompt, max_tokens=4000, temperature=0.4):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": "claude-3-haiku-20240307",
        "max_tokens": max_tokens,
        "temperature": temperature,
        "system": "You are a world-class researcher. Analyze the given information and generate a detailed report.",
        "messages": [{"role": "user", "content": prompt}],
    }
    try:
        response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
        response.raise_for_status()
        response_json = response.json()
        print(response_json)
        response_text = response_json['content'][0]['text']
        print(remove_first_line(response_text.strip()))
        return remove_first_line(response_text.strip())
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")
def search_web(search_term, api_key, cse_id):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        result = service.cse().list(q=search_term, cx=cse_id).execute()
        parsed_data = json.loads(json.dumps(result))
        relevant_text = []
        for item in parsed_data['items']:
            title = item['title']
            snippet = item.get('snippet', '')
            relevant_text.append(f"Title: {title}\nSnippet: {snippet}\n")
        print("\n".join(relevant_text))
        return relevant_text
    except Exception as e:
        print(f"Error occurred during web search: {e}")
        return []
def generate_subtopic_report(research_topic, subtopic):
    search_data = []
    all_queries = []
    search_cache = {}
    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 3 to 5 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
 
    initial_queries_response = generate_text(initial_queries_prompt)
    if initial_queries_response.startswith('[') and ']' in initial_queries_response:
        try:
            initial_queries = ast.literal_eval(initial_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping initial queries.")
            initial_queries = []
    else:
        print("Error: Search query format not found. Skipping initial queries.")
        initial_queries = []
   
    print(initial_queries)
    all_queries.extend(initial_queries)
    def search_and_cache(query):
        if query in search_cache:
            return search_cache[query]
        else:
            search_results = search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)
            search_cache[query] = search_results
            return search_results
    with concurrent.futures.ThreadPoolExecutor() as executor:
        search_results = list(executor.map(search_and_cache, initial_queries))
        search_data.extend(search_results)
 
    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt = f"Generate a detailed report on the subtopic '{subtopic}' of '{research_topic}' based on this search data:\n\n{str(search_data)}\n\nUse Markdown formatting."
    report = generate_text(report_prompt, max_tokens=4000)
    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt = f"Identify areas needing more detail in this '{subtopic}' report:\n\n{report}\n\nSearch queries so far:\n{str(all_queries)}\n\nGenerate 2-4 new search queries to fill in gaps."
    additional_queries_response = generate_text(analysis_prompt)
    if additional_queries_response.startswith('[') and ']' in additional_queries_response:
        try:
            additional_queries = ast.literal_eval(additional_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping additional queries.")
            additional_queries = []
    else:
        print("Error: Search query format not found. Skipping additional queries.")
        additional_queries = []
   
    all_queries.extend(additional_queries)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        additional_search_results = list(executor.map(search_and_cache, additional_queries))
        search_data.extend(additional_search_results)
    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt = f"Update this '{subtopic}' report with the new search data, keeping existing info and adding new details:\n\n{report}\n\nNew data:\n{str(additional_search_results)}\n\nUse Markdown."
    report = generate_text(update_prompt, max_tokens=4000)
    print(f"Final report generated for subtopic: {subtopic}!")
    return report
def generate_comprehensive_report(research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt = f"Combine these reports on subtopics of '{research_topic}' into a comprehensive overview:\n\n{subtopic_reports}\n\nInclude all key info in a detailed, well-structured report with a conclusion. Use Markdown and a table of contents."
    comprehensive_report = generate_text(comprehensive_report_prompt, max_tokens=4000)
    while "## Conclusion" not in comprehensive_report:
        print("Report is cut short. Generating additional content...")
        continuation_prompt = f"The comprehensive report on '{research_topic}' was cut short. Please continue generating it from here:\n\n{comprehensive_report}"
        additional_content = generate_text(continuation_prompt, max_tokens=4000)
        comprehensive_report += "\n\n" + additional_content
    print("Comprehensive report generated!")
    return comprehensive_report
research_topic = input("Enter the research topic: ")
# Generate subtopic checklist
subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist in a Python-parseable list. Return nothing but the list. Do so in one line. Maximum five sub-topics. Start your response with [\""
subtopic_checklist = ast.literal_eval('[' + generate_text(subtopic_checklist_prompt).split('[')[1])
print(f"Subtopic Checklist: {subtopic_checklist}")
subtopic_reports = []
for subtopic in subtopic_checklist:
    subtopic_report = generate_subtopic_report(research_topic, subtopic)
    subtopic_reports.append(subtopic_report)
comprehensive_report = generate_comprehensive_report(research_topic, "\n\n".join(subtopic_reports))
os.makedirs("reports", exist_ok=True)
report_filename = f"{research_topic.replace(' ', '_')}_report.md"
report_filepath = os.path.join("reports", report_filename)
try:
    with open(report_filepath, "w", encoding="utf-8") as file:
        file.write(comprehensive_report)
    print(f"Comprehensive report saved as '{report_filepath}'.")
except UnicodeEncodeError as e:
    print(f"Error: Unable to save the report due to unsupported characters. Please try again with a different research topic or modify the generated report to remove any unsupported characters.")
    print(f"Error details: {e}")

# Comprehensive Overview of Acoustic Guitars

## Table of Contents
1. [History of Acoustic Guitars](#history-of-acoustic-guitars)
2. [Types of Acoustic Guitars](#types-of-acoustic-guitars)
3. [Construction and Materials of Acoustic Guitars](#construction-and-materials-of-acoustic-guitars)
4. [Acoustic Guitar Brands and Manufacturers](#acoustic-guitar-brands-and-manufacturers)
5. [Acoustic Guitar Maintenance and Care](#acoustic-guitar-maintenance-and-care)
6. [Conclusion](#conclusion)

## History of Acoustic Guitars

### Ancient Beginnings
The origins of the acoustic guitar can be traced back to ancient civilizations, such as the Egyptians and Greeks, who used similar stringed instruments like the lyre and the kithara. These early instruments were often made of wood and had a flat base, with strings of equal length.

### The Development of the Guitar
The modern guitar evolved from the oud and the lute, which predate written history. The earliest known guitar-like instrument is a tanbur from Egypt, dating back to around 1500 BCE. Over the centuries, the design of the guitar continued to evolve, with various cultures and regions contributing to its development.

### The 19th Century and the Spanish Guitar
In the 19th century, Spanish musician Antonio de Torres Jurado created an instrument that was shaped similarly to today's acoustic guitars. He took the traditional guitar design and made improvements that enhanced the tone and volume of the instrument.

### The Age of Steel Strings
In the late 1800s, silk strings with steel wrapping became available, leading to the development of the steel-string acoustic guitar. This new design allowed for a louder and more powerful sound, which was particularly useful for accompanying vocals and other instruments.

### The Flat-Top Acoustic Guitar
The flat-top acoustic guitar, with its distinctive soundhole and flat top, became the most popular form of acoustic guitar in the early 20th century. Luthiers like Christian Frederick Martin, a German-born American, played a significant role in the evolution of this design.

### Innovations and Advancements
Throughout the 20th and 21st centuries, acoustic guitar design has continued to evolve, with various innovations and advancements. These include the development of the acoustic-electric guitar, the use of new materials and construction techniques, and the incorporation of advanced technologies like Yamaha's TransAcoustic technology.

### Recent Trends and Developments
In recent years, the acoustic guitar market has seen several new trends and developments, including a focus on sustainability and eco-friendly designs, the rise of hybrid designs, the integration of smart technology, increased personalization and customization, and the growth of niche and specialty guitars.

## Types of Acoustic Guitars

### Dreadnought
- The dreadnought is the most common and popular acoustic guitar body shape.
- It is known for its large size, which produces a powerful, loud, and bass-heavy tone.
- Dreadnoughts are often used for strumming and playing chords.

### Orchestra Model (OM) / Auditorium
- Also known as the "000" or auditorium shape, the OM is slightly smaller than a dreadnought.
- It has a more balanced tone with good volume and dynamic range.
- The OM is versatile and well-suited for fingerstyle playing as well as strumming.

### Parlor
- Parlor guitars are the smallest of the common acoustic guitar body shapes.
- They have a more intimate, mellow, and focused tone.
- Parlor guitars are often used for fingerstyle playing and are popular for their portability.

### Jumbo
- Jumbo acoustic guitars are characterized by their large, wide body shape.
- They are known for producing a powerful, booming sound with a strong low-end response.
- Jumbo guitars are often used for playing chords and strumming in genres like folk, country, and rock.

### Classical/Spanish
- Classical or Spanish acoustic guitars have a distinctive nylon-string construction.
- They are designed for fingerstyle playing and are commonly used in classical, flamenco, and Latin music styles.
- Classical guitars typically have a smaller, more delicate body shape compared to steel-string acoustics.

### 12-String
- 12-string acoustic guitars have twice the number of strings as a standard 6-string guitar.
- They produce a rich, full-bodied sound with a shimmering, chiming quality.
- 12-string guitars are popular for their unique sonic character and are often used in folk, rock, and worship music.

### Acoustic-Electric
- Acoustic-electric guitars are equipped with built-in pickups and preamps, allowing them to be amplified and played through a PA system or amplifier.
- They combine the natural, organic tone of an acoustic guitar with the versatility of electronic amplification.
- Acoustic-electric guitars are widely used in live performances, recording studios, and amplified settings.

## Construction and Materials of Acoustic Guitars

### Materials Used in Acoustic Guitar Construction
- **Tonewoods**: The most common tonewoods used for the back and sides of acoustic guitars include rosewood, mahogany, sapele, maple, koa, walnut, and sycamore.
- **Tops/Soundboards**: The top or soundboard of an acoustic guitar is often made of cedar or spruce.
- **Other Materials**: Bracing materials, fretboard and bridge materials, strings, and laminates/composites are also used in acoustic guitar construction.

### Acoustic Guitar Construction Techniques
- Lamination: Laminating the guitar sides is a common practice to help prevent deformation over time.
- Bracing: The bracing pattern and materials used for the guitar top and back play a crucial role in the instrument's tone and structural integrity.
- Neck Joint: The design and construction of the neck joint, where the neck meets the body, is an important consideration for guitar builders.
- Jigs and Fixtures: The use of specialized jigs and fixtures is common in acoustic guitar construction, as they help achieve precision and consistency.

### Factors Affecting Acoustic Guitar Sound
- Wood Composition: The choice of tonewoods for the top, back, and sides, as well as the bracing materials, can significantly impact the guitar's tone, volume, and overall sound character.
- String Materials: The composition of the guitar strings, including the core and winding materials, can affect the tonal qualities, such as brightness, warmth, and sustain.
- Environmental Factors: The climate and environmental conditions can impact the stability and performance of an acoustic guitar, particularly with regards to the wood materials.

## Acoustic Guitar Brands and Manufacturers

### Top Recommended Brands
1. Taylor
2. Breedlove and Bedell
3. Yamaha
4. Furch/Stonebridge
5. Martin

### Other Notable Brands
- Gibson
- Guild
- Alvarez
- Epiphone
- Takamine
- Godin (Seagull)

### Discontinued Brands
- Tacoma (previously owned by Fender and Guild)

### Manufacturing and Neck Joints
- Bolt-on neck joints can help strengthen the joint and improve tone and resonance.
- Large Chinese manufacturers produce guitars for other major brands, though the quality and reputation may vary.

## Acoustic Guitar Maintenance and Care

### Proper Humidity and Storage
- Maintain ideal humidity levels between 40-60%.
- Store your guitar in its case when not in use.
- Loosen the strings slightly when storing the guitar for long periods.
- Avoid exposing your guitar to extreme temperature and humidity fluctuations.

### Cleaning and Conditioning
- Wipe down the guitar's body, neck, and strings with a soft, dry cloth after each use.
- Use a guitar-specific cleaning solution and a microfiber cloth for periodic cleaning and conditioning.
- Avoid using household cleaners or abrasive materials.

### Maintenance and Adjustments
- Check the guitar's truss rod periodically and adjust it as needed.
- Adjust the saddle height to ensure optimal string action and playability.
- Tighten any loose tuning machines or other hardware components.
- Consider having your guitar professionally set up or serviced if needed.

### Accessories and Tools
- Use a humidifier to maintain proper humidity levels.
- Invest in a guitar maintenance kit with tools for string changes, fretboard conditioning, and basic repairs.
- Consult repair manuals or seek guidance from experienced luthiers for more complex tasks.

### Preventive Care
- Avoid exposing your guitar to excessive moisture, heat, or direct sunlight.
- Handle your guitar with care, avoiding sudden impacts or excessive bending of the neck.
- Use a guitar stand or wall mount to store your instrument when not in use.
- Regularly inspect your guitar for any signs of wear or damage.
- Adjust humidity levels more frequently if you live in a particularly dry or humid climate.

## Conclusion

The history of the acoustic guitar is a rich and diverse one, spanning centuries and encompassing numerous cultures and regions. From its ancient beginnings to the modern innovations, the acoustic guitar has remained a beloved and iconic musical instrument, constantly evolving to meet the needs of musicians and music enthusiasts alike.

The various types of acoustic guitars, ranging from the powerful dreadnought to the intimate parlor, offer players a wide range of tonal and physical characteristics to choose from. The construction and materials used in these instruments, as well as the techniques employed by luthiers, play a crucial role in shaping the overall sound and performance of an acoustic guitar.

The acoustic guitar market is also home to a diverse array of renowned brands and manufacturers, each with their own unique strengths and specialties. As the market continues to evolve, we can expect to see even more exciting developments in the world of acoustic guitars.

Proper maintenance and care are essential for ensuring the longevity and optimal performance of an acoustic guitar. By following the guidelines outlined in this report, guitar owners can help protect their instruments and maintain their tonal qualities for years to come.

Overall, the world of acoustic guitars is a rich and fascinating one, filled with history, innovation, and a deep appreciation for the craft and artistry of instrument-making. Whether you're a seasoned musician or a passionate hobbyist, the acoustic guitar remains a timeless and captivating musical companion.